In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html'

### Extract date

In [3]:
content = requests.get(url).content
soup =BeautifulSoup(content, 'lxml')

In [4]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

data = soup.findAll(text=re.compile(".*pdated .*202"))
data


['Last updated on May 8, 2020']

In [5]:
if len(data)>0:
    m = re.search(".*pdated .* (\w+ \d+, 202\d)", _removeNonAscii(data[0]))
    data_date_str = m.group(1)
    data_date = datetime.strptime(data_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
else:
    data_date = datetime.utcnow().strftime('%Y-%m-%d')
data_date

'2020-05-08'

### Extract data and add into Dataframe

In [6]:
data = soup.findAll(text=re.compile("Total Cases = (\d+).*Total Deaths = (\d+)"))
data

['[{"@context":"https:\\/\\/schema.org","@type":"SpecialAnnouncement","name":"Cases in the U.S.","text":"<h3>U.S. At A Glance<\\/h3>\\r\\n<ul>\\r\\n \\t<li>Total Cases = 1,248,040\\r\\n<ul>\\r\\n \\t<li>New Cases = 28,974<\\/li>\\r\\n<\\/ul>\\r\\n<\\/li>\\r\\n \\t<li>Total Deaths = 75,477\\r\\n<ul>\\r\\n \\t<li>New Deaths = 2,180<\\/li>\\r\\n<\\/ul>\\r\\n<\\/li>\\r\\n<\\/ul>","category":"https:\\/\\/www.wikidata.org\\/wiki\\/Q81068910","datePosted":"2020-05-08T18:10","spatialCoverage":{"type":"Country","name":"USA"},"diseasePreventionInfo":"https:\\/\\/www.cdc.gov\\/coronavirus\\/2019-ncov\\/prevent-getting-sick\\/index.html","diseaseSpreadStatistics":"https:\\/\\/www.cdc.gov\\/coronavirus\\/2019-ncov\\/covid-data\\/covidview\\/index.html"}]']

In [7]:

m = re.search("Total Cases = ([,\d]+).*Total Deaths = ([,\d]+)", _removeNonAscii(data[0]))


In [8]:
total_cases = int(m.group(1).replace(',',''))
total_cases

1248040

In [9]:
total_deaths = int(m.group(2).replace(',',''))
total_deaths

75477

In [10]:
df = pd.DataFrame({'datetime':[data_date], 
                   'total_cases': [total_cases],
                   'total_deaths': [total_deaths]})

In [11]:
df

,datetime,total_cases,total_deaths
0,2020-05-08,1248040,75477


### Save to file (old data)


In [12]:
from pathlib import Path
csv_file = '../data/us-cdc-total-cases-deaths.csv'

if Path(csv_file).exists():
    # read out the old dat
    old_df = pd.read_csv(csv_file)
    df = df.append(old_df)
df.drop_duplicates(subset=['datetime'], inplace=True)
df

,datetime,total_cases,total_deaths
0,2020-05-08,1248040,75477
0,2020-05-07,1219066,73297


In [13]:
df.sort_values(by=['datetime'], inplace=True)
df.to_csv(csv_file, index=False)

In [14]:
df

,datetime,total_cases,total_deaths
0,2020-05-07,1219066,73297
0,2020-05-08,1248040,75477
